In [1]:
# if using conda, conda install -c conda-forge opencv
import cv2
from xml.dom import minidom
import re
import os
folder = "PatRec17_KWS_Data"

In [57]:
def svg_to_img(lst):
    train = open(folder+"/task/train.txt").read().splitlines()
    valid = open(folder+"/task/valid.txt").read().splitlines()
    train_path = "data/train"
    valid_path = "data/valid"
    for img in lst:
        print("currently working on file "+img)
        dest_path = ''
        if img in train:
            dest_path = train_path
        if img in valid:
            dest_path = valid_path
        if not os.path.exists(dest_path):
            os.makedirs(dest_path)
        doc = minidom.parse(folder + "/ground-truth/locations/"+img+".svg")  # parseString also exists
        path_strings = [path.getAttribute('d') for path
                        in doc.getElementsByTagName('path')]
        ids_strings = [path.getAttribute('id') for path
                        in doc.getElementsByTagName('path')]
        doc.unlink()
        i = 0
        for path in path_strings:
            wordList = re.sub(" ", " ",  path).split()
            x = list(map(float,wordList[2::3]))
            y = list(map(float,wordList[1::3]))
            x = [round(el) for el in x]
            y = [round(el) for el in y]
            pts = []
            j = 0
            for el in x:
                pts.append([y[j],el])
                j = j + 1
            pts = np.array(pts)
            img2 = cv2.imread(folder + "/images/"+img+".jpg",0)
            ret, imgf = cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            ## (1) Crop the bounding rect
            rect = cv2.boundingRect(pts)
            x,y,w,h = rect
            croped = imgf[y:y+h, x:x+w].copy()

            ## (2) make mask
            pts = pts - pts.min(axis=0)

            mask = np.zeros(croped.shape[:2], np.uint8)
            cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

            ## (3) do bit-op
            dst = cv2.bitwise_and(croped, croped, mask=mask)

            ## (4) add the white background
            bg = np.ones_like(croped, np.uint8)*255
            cv2.bitwise_not(bg,bg, mask=mask)
            dst2 = bg+ dst
            resized_image = cv2.resize(dst2, (150, 75)) 
            cv2.imwrite(os.path.join(dest_path , ids_strings[i]+'.jpg'), resized_image)
            i = i + 1

In [58]:
lst = ['270','271','272','273','274','275','276','277','278','279','300','301','302','303','304']
svg_to_img(lst)

currently working on file 270
currently working on file 271
currently working on file 272
currently working on file 273
currently working on file 274
currently working on file 275
currently working on file 276
currently working on file 277
currently working on file 278
currently working on file 279
currently working on file 300
currently working on file 301
currently working on file 302
currently working on file 303
currently working on file 304


In [53]:
def get_mean_ratio():
    lst = ['270','271','272','273','274','275','276','277','278','279','300','301','302','303','304']
    train = open(folder+"/task/train.txt").read().splitlines()
    valid = open(folder+"/task/valid.txt").read().splitlines()
    train_path = "data/train"
    valid_path = "data/valid"
    ratio = 0.0
    k = 0
    for img in lst:
        print("currently working on file "+img)
        dest_path = ''
        if img in train:
            dest_path = train_path
        if img in valid:
            dest_path = valid_path
        if not os.path.exists(dest_path):
            os.makedirs(dest_path)
        doc = minidom.parse(folder + "/ground-truth/locations/"+img+".svg")  # parseString also exists
        path_strings = [path.getAttribute('d') for path
                        in doc.getElementsByTagName('path')]
        ids_strings = [path.getAttribute('id') for path
                        in doc.getElementsByTagName('path')]
        doc.unlink()
        i = 0
        for path in path_strings:
            wordList = re.sub(" ", " ",  path).split()
            x = list(map(float,wordList[2::3]))
            y = list(map(float,wordList[1::3]))
            x = [round(el) for el in x]
            y = [round(el) for el in y]
            pts = []
            j = 0
            for el in x:
                pts.append([y[j],el])
                j = j + 1
            pts = np.array(pts)
            img2 = cv2.imread(folder + "/images/"+img+".jpg",0)
            ret, imgf = cv2.threshold(img2, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            ## (1) Crop the bounding rect
            rect = cv2.boundingRect(pts)
            x,y,w,h = rect
            croped = imgf[y:y+h, x:x+w].copy()

            ## (2) make mask
            pts = pts - pts.min(axis=0)

            mask = np.zeros(croped.shape[:2], np.uint8)
            cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

            ## (3) do bit-op
            dst = cv2.bitwise_and(croped, croped, mask=mask)

            ## (4) add the white background
            bg = np.ones_like(croped, np.uint8)*255
            cv2.bitwise_not(bg,bg, mask=mask)
            dst2 = bg+ dst
            ratio = ratio + (len(dst2)/len(dst2[0]))
            k = k + 1
            i = i + 1
    print(ratio/k)

In [54]:
# no more needed, it was used to have a good resize
# get_mean_ratio() 

currently working on file 270
currently working on file 271
currently working on file 272
currently working on file 273
currently working on file 274
currently working on file 275
currently working on file 276
currently working on file 277
currently working on file 278
currently working on file 279
currently working on file 300
currently working on file 301
currently working on file 302
currently working on file 303
currently working on file 304
0.49735874800468355
